This tutorial gives an overview of the interaction with the *PowerFactory* database using *powfacpy*. The methods introduced in this tutorial serve as a basis to do more complex things and to write more readable code using less lines of code. Furthermore, the error messages *powfacpy* provides are a great help when debugging larger scripts. 

For a complete list of classes and methods, please have a look at the *API* Chapter of the documentation or at the source code.

Similar to using the Python API of *PowerFactory* directly, we first need to import the powerfactory module from *PowerFactory*'s installation directory (in case you want to access PowerFactory externally using IPython/Jupyter) and get the application. 

In [1]:
# If you use IPython/Jupyter:
import sys
sys.path.append(r'C:\Program Files\DIgSILENT\PowerFactory 2022 SP1\Python\3.10') # you may use a different directory
# Get the PF app
import powerfactory
app = powerfactory.GetApplication()

Then import *powfacpy* and create an instance of class `PFBaseInterface` (with argument `app`). This interface class is used mainly to interact with the *PowerFactory* database as we will see below. 

In [2]:
import powfacpy
pfbi = powfacpy.PFBaseInterface(app)

Note that `pfbi` has an attribute `app` that can be used similar to the `app` variable we loaded from the `powerfactory` module before. Here are two ways to 1. show the PowerFactory application and 2. activate a project:

In [3]:
app.Show()
app.ActivateProject(r"\seberlein\powfacpy\powfacpy_tests") # You may change the project path. 
# Make sure the path starts with "\"
pfbi.app.Show()
pfbi.app.ActivateProject(r"\seberlein\powfacpy\powfacpy_tests") # You may change the project path.
# Make sure the path starts with "\"

0

So where does `pfbi` differ from the Python interface that `app` provides? Let's see.

# Accessing objects
 Let's access an object from the *PowerFactory* database. When using `app`, we would use the methods `app.GetProjectFolder`, `app.GetChildren` or `app.GetContents`. Instead, we use `pfbi.get_obj` with the path of the object (**relative to the folder of the active project**) as an argument: 

In [4]:
mv_terminal = pfbi.get_obj(r"Network Model\Network Data\test_base_interface\Grid\Terminal MV")[0]

The `[0]` at the end is necessary because the method always returns a list (with one element in this case). 

Note that you can easily copy and paste the path from the data manager while selecting the object in the data tree: 

![object path](figures/object_path.png)

`pfbi.get_obj` also differs in other ways from `app.GetContents`. You can set a condition for the objects that you want to access, which is best described by an example:

In [5]:
hv_terminals = pfbi.get_obj(r"Network Model\Network Data\test_base_interface\Grid\*.ElmTerm",
        condition=lambda x: getattr(x,"uknom") > 50)

First of all, we use a wildcard (`*.ElmTerm`) to access all terminals in the `Grid` folder. The `condition` argument is a function that defines a certain condition the objects have to fulfill and returns a boolean value. In this case, the input argument `x` is an `ElmTerm` and `getattr(x,"uknom") > 50` gets its attribute `uknom` (nominal voltage) and checks whether it is larger than 50 (kV). `getattr` is a pthon method that returns an attribute of any object (also *PowerFactory* objects). You can also define more complex functions, such as `lambda x: getattr(x,"uknom") > 50 and getattr(x,"uknom") < 200` to access terminals between 50 and 200 kV.

It is also possible to include subfolders in the search for objects:

In [6]:
terminals = pfbi.get_obj(r"Network Model\Network Data\*.ElmTerm",include_subfolders=True)

This will search in `Network Model\Network Data` and all its subfolders.

What's also very helpful is that `pfbi.get_obj` throws an error if the path is incorrect and shows exactly where it fails. So

`terminals = pfbi.get_obj(r"Network Model\wrong_folder_name\*.ElmTerm")` 

will throw an error:

PFPathError: 'wrong_folder_name' does not exist in '\user\powfacpy_base\Network Model'

By default, an exception is also raised if no objects are found in the specified folder. For example:

```python
terminals = pfbi.get_obj(r"Network Model\Network Data\wrong_object_name*",include_subfolders=True)
```

returns

PFPathError: 'wrong_object_name*' does not exist in '\user\powfacpy_base\Network Model\Network Data'

This can be turned off using `error_if_non_existent=False`, then an empty list will be returned.

If you want to access objects in a folder many times and don't want to use every time the whole path relative to the active project, you can also specifiy a parent folder where the `path` starts (this can also be more performant):

In [7]:
grid_folder = pfbi.get_obj(r"Network Model\Network Data\test_base_interface\Grid")[0]
mv_terminal = pfbi.get_obj("Terminal MV",   parent_folder=grid_folder)[0]
hv_terminal = pfbi.get_obj("Terminal HV 1", parent_folder=grid_folder)[0]

An alternative to `pfbi.get_obj` is `pfbi.get_single_obj`. Use this function if you want to access a single unique object from the database and want to avoid the `[0]` (which is easily forgotten):

In [8]:
terminal_1=pfbi.get_single_obj(r"Network Model\Network Data\test_base_interface\Grid\Terminal HV 1") 

# Setting and getting object attributes
How about setting data in the database? You can set attributes of an object as follows:

In [9]:
pfbi.set_attr(r"Network Model\Network Data\test_base_interface\Grid\Terminal MV",
    {"uknom":33,"outserv":0})

So with only one command we set the attributes "uknom" and "outserv" of the terminal. This saves time and is also very readable code!
Note that the method `set_attr` accepts the path (string) but also a *PowerFactory* object. For example, we could also use the object `mv_terminal` that we loaded above:

In [10]:
pfbi.set_attr(mv_terminal, {"uknom":33,"outserv":0})

This applies to many other methods in `powfacpy`. Loading the object only once and then using the object can be more efficient than using the path string very many times.

If you want to get an attribute of an object, write: 

In [11]:
nominal_voltage = pfbi.get_attr(r"Network Model\Network Data\test_base_interface\Grid\Terminal MV","uknom")
# Equivalent:
nominal_voltage = pfbi.get_attr(mv_terminal,"uknom")
nominal_voltage = pfbi.get_attr("Terminal MV","uknom",parent_folder=r"Network Model\Network Data\test_base_interface\Grid")

# Creating objects
If you want to create a new object, use:

In [12]:
new_obj = pfbi.create_by_path(r"Library\Dynamic Models\dummy.BlkDef") 

This will create an object of class "BlkDef" with the name "dummy" in the folder "Library\Dynamic Models". You can also use

In [13]:
new_obj = pfbi.create_in_folder(r"Library\Dynamic Models","dummy.BlkDef",overwrite=True)
# Or equivalent:
models_folder = pfbi.get_single_obj(r"Library\Dynamic Models")
new_obj = pfbi.create_in_folder(models_folder,"dummy.BlkDef",overwrite=True)

which will overwrite the former object (this avoids that *PowerFactory* creates *dummy(1), dummy(2),...* everytime the code is run).

# Copying objects
The method `copy_obj` can be seen as a method that uses `get_obj` as a first step to access objects and then copies them to a target folder under the hood. Therefore, `copy_obj`shares many (optional) input arguments with `get_obj`. Additionally, a target folder is required. For example, to copy all objects from a source folder to a target folder, write

In [14]:
source_folder = r"Library\Dynamic Models\TestDummyFolder"
target_folder = r"Library\Dynamic Models\TestCopyMultiple"
new_copied_objects = pfbi.copy_obj("*",target_folder,parent_folder=source_folder)

Note that `parent folder` is an optional argument (used for the **source**), so you could also specify the full object path as the first argument instead of using `*` (similar to `get_obj`). To select certain objects from the source folder, you can use optional arguments like `condition` or `include_subfolders`, also similar to the `get_obj` method. The method also accepts a list of *PowerFactory* objects:

In [15]:
list_of_objects_to_copy = pfbi.get_obj("*",parent_folder=source_folder)
target_folder_obj = pfbi.get_single_obj(r"Library\Dynamic Models\TestCopyMultiple")
new_copied_objects = pfbi.copy_obj(list_of_objects_to_copy,
    target_folder_obj, overwrite=True) 

Note that the target folder object (instead of a path string) is used (second argument).

If you want to copy a single object, use `copy_single_obj`. You have the option to give it a new name:

In [16]:
new_copy = pfbi.copy_single_obj(r"Library\Dynamic Models\Linear_interpolation",
    r"Library\Dynamic Models\TestCopy", new_name="new_dummy_name")

First input here is the path of the object to be copied, but you can also use a *PowerFactory* object.

# Deleting objects
Deleting objects works in a similar fashion, because under the hood it uses `get_obj` and deletes the returned objects in the second step.

In [17]:
pfbi.delete_obj("to_be_deleted*",
    parent_folder=r"Library\Dynamic Models",
    include_subfolders=True,
    error_if_non_existent=False)

And of course you could also use *PowerFactory* objects as an input:

In [18]:
objects_in_folder = pfbi.get_obj("to_be_deleted*",parent_folder=r"Library\Dynamic Models",
    error_if_non_existent=False)
pfbi.delete_obj(objects_in_folder)